In [ ]:
%matplotlib widget
import networkx
from typing import List, Tuple
import matplotlib.pyplot
import numpy

# Setup

## Specify Graph

In [ ]:
edges: List[Tuple[int, int, float]] = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (1, 2, 1.0),
    (1, 3, 1.0),
    (2, 3, 1.0),
]

In [ ]:
graph = networkx.Graph()
graph.add_weighted_edges_from(edges)
N = graph.number_of_nodes()

In [ ]:
positions = networkx.spring_layout(graph, fixed=[0,3], pos={0: (-0.5, 0), 3: (0.5, 0)})
networkx.draw_networkx_nodes(graph, positions)
networkx.draw_networkx_edges(graph, positions)
networkx.draw_networkx_labels(graph, positions, font_color="white")
networkx.draw_networkx_edge_labels(graph, positions, edge_labels=networkx.get_edge_attributes(graph, 'weight'))
matplotlib.pyplot.gcf().set_size_inches(3.2, 2.4)
matplotlib.pyplot.gcf().set_tight_layout(True)

# export SVG
#cut = 1.2
#xmax = cut * max(x for x, _ in positions.values())
#ymax = cut * max(y for _, y in positions.values())
#xmin = cut * min(x for x, _ in positions.values())
#ymin = cut * min(y for _, y in positions.values())
#matplotlib.pyplot.gca().set_xlim(xmin, xmax)
#matplotlib.pyplot.gca().set_ylim(ymin, ymax)
#matplotlib.pyplot.savefig("graph.svg")
#matplotlib.pyplot.close()


matplotlib.pyplot.show()

## Setup Problem for Qiskit

First we obtain the weight matrix of the graph:

In [ ]:
weight_matrix = networkx.convert_matrix.to_numpy_array(graph)
weight_matrix

Qiskit provides a handy routine to obtain the Ising Hamiltonian associated with the Maximum-Cut problem. It returns a weighted Ising operator and an energy offset from the constant term.

In [ ]:
from qiskit.optimization.applications.ising import max_cut

hamiltonian, offset = max_cut.get_operator(weight_matrix)
print("Hamiltonian:")
print("------------")
print(hamiltonian)
print("energy offset:", offset)
print(hamiltonian.print_details())

# Solve the Problem

In [ ]:
def plot_solution(solution: List[int]):
    colors = ["C0" if solution[i] else "C1" for i in range(N)]
    networkx.draw_networkx_nodes(graph, positions, node_color=colors)
    networkx.draw_networkx_edges(graph, positions)
    networkx.draw_networkx_labels(graph, positions, font_color="white")
    networkx.draw_networkx_edge_labels(graph, positions, edge_labels=networkx.get_edge_attributes(graph, 'weight'))

## Brute-Force

In [ ]:
best_profit = 0.

for combination in range(2**N):
    # generate solution candidates (lists of 0's and 1's):
    # 1. bin() converts to binary string
    # 2. [:2] removes the '0b' prefix
    # 3. .zfill(N) prepends 0s until a length of N has been achieved
    binary = [int(digit) for digit in bin(combination)[2:].zfill(N)]
    
    # evaluate the cost function
    profit = 0.0
    for i in range(N):
        for j in range(N):
            profit += weight_matrix[i, j] * binary[i] * (1-binary[j])
    
    # check if we found a better solution
    if profit > best_profit:
        best_profit = profit
        solution = binary
    
    # print info about current combination
    print("combination {}: binary = {}, profit = {}".format(combination, str(binary), profit))

print()
print("optimal solution: binary = {}, profit = {}".format(str(solution), best_profit))

plot_solution(solution)

In [ ]:
plot_solution(solution)